#### This notebook is where we will extract the pertinent weather data for the flights TEST data set

In [1]:
# Import pandas for df manipulations
import pandas as pd 

from functions import get_hour, time_of_day 

In [2]:
# Load the datasets
weather = pd.read_csv('data/WeatherEvents_Jan2016-Dec2020.csv')
fl_data = pd.read_csv('data/flights_test.csv', low_memory = False)

In [3]:
# Inspect the data dataframe
fl_data.head()

,Unnamed: 0,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,...,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,...,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,...,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,...,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,...,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,...,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333


Need to make an hour column in data

In [4]:
# Create a column with the hour that the plane departs and arrives

fl_data['dep_hour'] = fl_data['crs_dep_time'].apply(get_hour)
fl_data['arr_hour'] = fl_data['crs_arr_time'].apply(get_hour)
fl_data['arr_hour'].dtypes

dtype('int64')

In [6]:
# Create a column that assigns a period of the day to a departure
fl_data['dep_period'] = fl_data['dep_hour'].apply(time_of_day)
fl_data['arr_period'] = fl_data['arr_hour'].apply(time_of_day)
fl_data['arr_period'].value_counts()

Afternoon    54106
Evening      50081
Morning      41772
Night         4664
Name: arr_period, dtype: int64

In [8]:
# Build the matchcode variable for the fl_data table
fl_data['origin_match_code'] = fl_data['origin'].astype(str) + fl_data['fl_date'].astype(str) + fl_data['dep_period'].astype(str)
fl_data['dest_match_code'] = fl_data['dest'].astype(str) + fl_data['fl_date'].astype(str) + fl_data['arr_period'].astype(str)

In [9]:
# Let's engineer some columns a little bit
weather['date'] = pd.DatetimeIndex(weather['StartTime(UTC)']).date
weather['hour'] = pd.DatetimeIndex(weather['StartTime(UTC)']).hour
weather['period'] = weather['hour'].apply(time_of_day)
weather = weather.drop(columns = ['StartTime(UTC)', 'EndTime(UTC)'])


In [10]:
# Remove the K prefix from the airport codes
weather['AirportCode'] = weather['AirportCode'].apply(lambda x: x[1:])

In [11]:
# Encode the match code column to match values to the data dataframe
weather['match_code'] = weather['AirportCode'].astype(str) + weather['date'].astype(str) + weather['period'].astype(str)

In [12]:
# Drop unnecessary columns

weather = weather.drop(columns = ['EventId',
                                  'TimeZone',
                                 'AirportCode',
                                 'LocationLat',
                                 'LocationLng',
                                 'City',
                                 'County',
                                 'State',
                                 'ZipCode'])

In [13]:
# Drop duplicate values
weather = weather.drop_duplicates(subset = 'match_code')
weather.shape

(3097656, 6)

In [14]:
# Create dictionaries for severity and condition with match_code as the key
severity_dict = dict(zip(weather['match_code'], weather['Severity']))
condition_dict = dict(zip(weather['match_code'], weather['Type']))

In [15]:
len(severity_dict)

3097656

In [17]:
# Use the previously created dictionaries to add severity and condition to the flights dataset
fl_data['origin_weather_condition'] = fl_data['origin_match_code'].map(condition_dict)
fl_data['origin_weather_severity'] = fl_data['origin_match_code'].map(severity_dict)
fl_data['dest_weather_condition'] = fl_data['dest_match_code'].map(condition_dict)
fl_data['dest_weather_severity'] = fl_data['dest_match_code'].map(severity_dict)
fl_data.head()

,Unnamed: 0,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,...,dep_hour,arr_hour,dep_period,arr_period,origin_match_code,dest_match_code,origin_weather_condition,origin_weather_severity,dest_weather_condition,dest_weather_severity
0,0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,...,18,19,Evening,Evening,ONT2020-01-01Evening,SFO2020-01-01Evening,NaN,NaN,NaN,NaN
1,1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,...,11,13,Morning,Afternoon,ONT2020-01-01Morning,SFO2020-01-01Afternoon,NaN,NaN,NaN,NaN
2,2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,...,20,21,Evening,Evening,ONT2020-01-01Evening,SJC2020-01-01Evening,NaN,NaN,NaN,NaN
3,3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,...,13,14,Afternoon,Afternoon,ONT2020-01-01Afternoon,SJC2020-01-01Afternoon,NaN,NaN,NaN,NaN
4,4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,...,9,10,Morning,Morning,ONT2020-01-01Morning,SJC2020-01-01Morning,NaN,NaN,NaN,NaN


In [18]:
# Replace NaN weather conditions with 'Clear' and severity with 'Light'
fl_data['origin_weather_condition'].fillna('Clear', inplace=True)
fl_data['origin_weather_severity'].fillna('Light', inplace=True)
fl_data['dest_weather_condition'].fillna('Clear', inplace=True)
fl_data['dest_weather_severity'].fillna('Light', inplace=True)
fl_data.head()


,Unnamed: 0,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,...,dep_hour,arr_hour,dep_period,arr_period,origin_match_code,dest_match_code,origin_weather_condition,origin_weather_severity,dest_weather_condition,dest_weather_severity
0,0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,...,18,19,Evening,Evening,ONT2020-01-01Evening,SFO2020-01-01Evening,Clear,Light,Clear,Light
1,1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,...,11,13,Morning,Afternoon,ONT2020-01-01Morning,SFO2020-01-01Afternoon,Clear,Light,Clear,Light
2,2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,...,20,21,Evening,Evening,ONT2020-01-01Evening,SJC2020-01-01Evening,Clear,Light,Clear,Light
3,3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,...,13,14,Afternoon,Afternoon,ONT2020-01-01Afternoon,SJC2020-01-01Afternoon,Clear,Light,Clear,Light
4,4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,...,9,10,Morning,Morning,ONT2020-01-01Morning,SJC2020-01-01Morning,Clear,Light,Clear,Light


In [ ]:
fl_data

In [20]:
# Write the data to a new csv
data.to_csv('data/flights_weather.csv')